# Create the Topography of the 79NG Fjord GETM Setup

This notebook creates the topography of the 79NG fjord in a format that is suitable for GETM.
The topography consists of the bathymetry, extracted from the RTopo dataset, and the ice thickness, extracted from BedMachine.
In addition, the position of the grounding line is determined, which is needed for the subglacial discharge.

A high resolution ($\Delta \text{lon}=0.025°$, $\Delta \text{lat} = 0.00415°$) and a low resolution ($\Delta \text{lon} = 0.050°$, $\Delta \text{lat} = 0.00830°$) are implemented.

By Markus Reinert, based on a notebook by Samira Zander, 2021.

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cmocean

In [ ]:
def show_topography(
    show_water_column=False,
    figure_kwargs={"figsize": (12, 4), "dpi": 200},
    bathy_plot_kwargs={},
    thick_plot_kwargs={},
):
    """Create a figure with 3 subplots showing the bottom and ice topography.
    
    The left plot shows the bedrock topogaphy and can be modified by
    passing a dictionary to bathy_plot_kwargs.

    The center plot shows the ice thickness (by default) or the
    approximated water column thickness (by setting show_water_column=True)
    and can be modified by passing a dictionary to thick_plot_kwargs.

    The right plot shows the approximated mask of ocean and land/ice.
    
    The figure (e.g., size and resolution) can be configured by
    passing a dictionary to figure_kwargs.

    Returns the Figure object and an array of 3 AxesSubplot.
    
    To approximate the draft of the ice tongue, which is needed for
    the water column thickness and the ocean mask, we use that the
    ice density is about 90 % of the ocean water density, implying
    that 90 % of floating ice is below sea level.
    """
    ice_draft = -0.9 * ice_thickness

    fig, axs = plt.subplots(ncols=3, sharex=True, sharey=True, constrained_layout=True, **figure_kwargs)

    ax = axs[0]
    if "cmap" not in bathy_plot_kwargs:
        bathy_plot_kwargs["cmap"] = cmocean.cm.deep_r
    bathymetry.plot(ax=ax, **bathy_plot_kwargs)
    ax.set_title("Bedrock topography around 79NG (RTopo)")

    ax = axs[1]
    if show_water_column:
        if "cmap" not in thick_plot_kwargs:
            thick_plot_kwargs["cmap"] = cmocean.cm.balance_r
        im = (ice_draft - bathymetry).plot(ax=ax, **thick_plot_kwargs)
        im.colorbar.set_label("Thickness [m]")
        ax.set_title("Water column thickness around 79NG")
    else:
        if "cmap" not in thick_plot_kwargs:
            thick_plot_kwargs["cmap"] = cmocean.cm.amp
        ice_thickness.plot(ax=ax, **thick_plot_kwargs)
        ax.set_title("Ice thickness around 79NG (BedMachine)")
    del thick_plot_kwargs["cmap"]

    ax = axs[2]
    im = (ice_draft > bathymetry).plot(ax=ax, cmap="Greys_r")
    im.colorbar.set_ticks([0, 1])
    im.colorbar.set_ticklabels(["land/ice", "ocean"])
    im.colorbar.set_label("")
    ax.set_title("Approximated ocean mask")

    for i, ax in enumerate(axs):
        ax.grid()
        if i > 0:
            ax.set_ylabel("")
    return fig, axs

## Load and show the data

Two datasets with the same coordinate system and the same resolution are needed.
They were created with QGIS.

In [ ]:
# Choose between: True False
HIGH_RESOLUTION = False

if HIGH_RESOLUTION:
    file_rtopo = "data/RTopo_WGS84_EPSG4326_res_0.025_0.00415.nc"
    file_bedmachine = "data/BedMachine_WGS84_EPSG4326_res_0.025_0.00415.nc"
else:
    file_rtopo = "data/RTopo_WGS84_EPSG4326_res_0.05_0.0083.nc"
    file_bedmachine = "data/BedMachine_WGS84_EPSG4326_res_0.05_0.0083.nc"

bathymetry = xr.open_dataset(file_rtopo).Band1
bathymetry.attrs["long_name"] = "Elevation"
bathymetry.attrs["units"] = "m"

ice_thickness = xr.open_dataset(file_bedmachine).Band1
ice_thickness.attrs["long_name"] = "Ice thickness"
ice_thickness.attrs["units"] = "m"

# Check that the grids of the datasets are the same
assert all(bathymetry.lon == ice_thickness.lon)
assert all(bathymetry.lat == ice_thickness.lat)

print(f"Grid size (lon×lat): {bathymetry.lon.size}×{bathymetry.lat.size} ({'high' if HIGH_RESOLUTION else 'low'})")

show_topography(bathy_plot_kwargs={"cmap": cmocean.cm.topo, "vmin": -bathymetry.max()});

## Crop the data to the model domain

Both datasets must be cropped to the same extent.
This is not yet the final extent.

In [ ]:
bathymetry = bathymetry.sel(lon=slice(-23, -15), lat=slice(79.2, 80.3))
ice_thickness = ice_thickness.sel(lon=slice(-23, -15), lat=slice(79.2, 80.3))

show_topography(bathy_plot_kwargs={"cmap": cmocean.cm.topo, "vmin": bathymetry.min()});

## Delete land cells

The ocean model only needs the data of water cells, so we can remove positive elevation values by replacing them with NaN.

In [ ]:
bathymetry = bathymetry.where(bathymetry < 0, np.NaN)
ice_thickness = ice_thickness.where(~np.isnan(bathymetry), np.NaN)

show_topography();

## Determine the grounding line

We need to know the grounding line position for two reasons.

1. To mask out grid points where the ice is grounded.
2. For the positions of the subglacial discharge.

For the grounding line, we use the coastline data of RTopo, limited to 79NG, since 79NG is the only glacier in our model domain.

In [ ]:
# Load the grounding line position from RTopo
gline_lon, gline_lat = np.loadtxt("data/RTopo-2.0.4_gl.asc").T

fig, axs = show_topography(show_water_column=True)
# To focus on the calving front, add thick_plot_kwargs={"vmax": 100} above and uncomment the following line:
# axs[0].set_xlim(-21, -18); axs[0].set_ylim(79.4, 79.75)

# Limit the data to the the area of 79NG (a coarse selection is sufficient)
select_79NG = (gline_lat > 79.2) & (gline_lon > -25)
gline_lon = gline_lon[select_79NG]
gline_lat = gline_lat[select_79NG]

for ax in axs:
    ax.plot(gline_lon, gline_lat, ".", color="orange", markersize=1, label="grounding line")

# Remove grounding line points at the calving front
# At the calving front, the water column is mostly > 100 m thick, so the ice does not seem to be grounded.
select_calving_front = (gline_lon > -19.9) & (gline_lat > 79.44) & (gline_lat < 79.65)
for ax in axs:
    ax.plot(gline_lon[select_calving_front], gline_lat[select_calving_front], "rx", markersize=2, label="removed points")
axs[1].legend(handlelength=0.5, loc="upper left")
axs[2].plot([-19.9, -19.1, -19.1, -19.9, -19.9], [79.44, 79.44, 79.65, 79.65, 79.44], "r", linewidth=1)
axs[2].text(-19, 79.5, "area in which\npoints are removed", color="r", size="small")

gline_lon = gline_lon[~select_calving_front]
gline_lat = gline_lat[~select_calving_front]

### Rasterize the grounding line

In [ ]:
gline = xr.zeros_like(bathymetry, dtype=bool)
for lon, lat in zip(gline_lon, gline_lat):
    nearest_gridpoint = gline.sel(lat=lat, lon=lon, method="nearest")
    gline.loc[{"lat": nearest_gridpoint.lat, "lon": nearest_gridpoint.lon}] = True
gline.plot()